# Website Recovery Procedure

 > You will review the steps within the documented procedure for remediation of website issues.

# Table of Contents
 - [Prerequisites](#Prerequisites)
   - [Install Modules](#install-modules)
   - [Testing Your Site](#testing-your-site)
 - [AWS Discovery](#aws-discovery)
   - [Review Route 53 Configuration](#review-route-53-configuration)
   - [Review Elastic Load Balancer (ELB) Configuration](#review-elastic-load-balancer-(ELB)-configuration)
   - [Review Elastic Compute Cloud (EC2) Configuration](#review-elastic-compute-cloud-(EC2)-configuration)
 - [Remediate](#remediate)
 - [Appendix](#appendix)

In [3]:
#!mermaid
flowchart TB
    subgraph one [Prerequisites]
        direction LR
        s1(Install Modules) --> s2(Testing Your Site)
    end
    subgraph two [AWS Discovery]
        direction LR
        s5(Review Route 53 Config) --> s6(Review ELB Config)
        s6 --> s7(Review EC2 Config)
    end
    one ==> two
    two ==> three(Remediate)

# Prerequisites

## Install Modules

You will use PowerShell to work with the [AWS Tools for PowerShell](https://aws.amazon.com/powershell/) and to do this you will require the use of a few modules.

Begin by:
 - Identifying the core required modules using the [array sub-expression operator](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_arrays#the-array-sub-expression-operator)

In [ ]:
$modules=@(
    "AWS.Tools.Common"
    "AWS.Tools.Installer"
)

* The AWS Tools for PowerShell will also use a separate installation process for the service specific modules

In [ ]:
$awsModules=@(
    "AWS.Tools.EC2"
    "AWS.Tools.ElasticLoadBalancingV2"
    "AWS.Tools.Route53"
)

 - Identifying currently available AWS Tools for PowerShell modules

> ---
> ✒️ **_NOTE:_** Below you will use the `?` alias for Where-Object and the `$_` alias for [`$PSItem`](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_psitem)
>
> ---

In [ ]:
$available = Get-Module -ListAvailable|?{$_.Name -like "AWS.Tools.*"}

 - Then if a necessary module is not available you will install it

In [ ]:
foreach($module in $modules)
{
    if($module -notin $available.Name)
    {
        Install-Module $module -Force
    }
}

foreach($module in $awsModules)
{
    if($module -notin $available.Name)
    {
        Install-AWSToolsModule $module -Force
    }
}

## Testing Your Site

Test to see if the public URL is reachable

In [ ]:
Invoke-WebRequest https://demo.soule.aws.sentinel.com

# AWS Discovery

> ---
> ℹ️ **_Information:_** Your JupyterLab instance runs as a Fargate task and inherits an Identity & Access Management (IAM) Role, so you do not need to explicitly authenticate.
>
> ---

## Review Route 53 Configuration

Now that you have the necessary modules, you can begin reviewing the existing environment. You will begin identifying resource dependencies from the outside in as you know the website's public URL.

 - Begin by looking up your Route 53 Zone

In [ ]:
$id = (Get-R53HostedZonesByName|?{$_.Name -like "soule.*"}).Id
$id

 - Then get the records within your Zone

In [ ]:
$sets = (Get-R53ResourceRecordSet $id).ResourceRecordSets
$sets|select -First 1

 - Find the record for your website

In [ ]:
$records = ($sets|?{$_.Name -like "demo.*"}).ResourceRecords.Value
$records

### Testing Using Record Value

 - With the currently configured source for your website URL test if the page can load

In [ ]:
$records|%{Invoke-WebRequest "https://$_" -SkipCertificateCheck}

## Review Elastic Load Balancer (ELB) Configuration

Next you will want to determine what resources are hosting the website.

 - Begin by finding the ELB that the DNS record references


In [ ]:
$lb = Get-ELB2LoadBalancer|?{$_.DnsName -in $records}
$lb

 - Now that you identified the load balancer you can determine what resources are in it's target group

In [ ]:
$group = Get-ELB2TargetGroup -LoadBalancerArn $lb.LoadBalancerArn
$group

 - Start by checking the health of the target group resources
   - You can also format the selection for easier review

In [ ]:
$targetHealth = Get-ELB2TargetHealth $group.TargetGroupArn
$targetHealth|ft @{n="target";e={$_.Target.Id}},@{n="health";e={$_.TargetHealth.State}}

 - Finally you will capture the supporting resources for the ELB

In [ ]:
$instances = (Get-EC2Instance|?{$_.Instances.InstanceId -in $targetHealth.Target.Id}).Instances
$instances

### Testing Using Targets

 - Test each of the supporting resources of the ELB using the health check properties for the target group and the internal addresses

In [ ]:
$instances|%{Invoke-WebRequest "$($group.HealthCheckProtocol)://$($_.PrivateIpAddress):$($group.Port)$($group.healthCheckPath)"}

## Review Elastic Compute Cloud (EC2) Configuration

Finally you will collect more information on the supporting resources before identifying the cause and remediating.

 - For each of your supporting resources gather their instance state

In [ ]:
$instanceState = Get-EC2InstanceStatus -InstanceIds $instances -IncludeAllInstances $true|select InstanceId,@{n="InstanceState";e={$_.InstanceState.Name}},@{n="InstanceStatus";e={$_.Status.Details}}
$instanceState

### Identify Stopped Targets

 - Identify if any of your instances are not in a running or pending state

In [ ]:
$stoppedInstances = $instanceState|?{$_.InstanceState -notin @("running","pending")}
$stoppedInstances

# Remediate

In this demo you can remediate the issue by starting the stopped instance.

In [ ]:
$stoppedInstances|%{Start-EC2Instance $_.InstanceId}

 - Validate your environment configuration and test your different dependency steps

In [ ]:
$instanceState = Get-EC2InstanceStatus -InstanceIds $instances -IncludeAllInstances $true|select InstanceId,@{n="InstanceState";e={$_.InstanceState.Name}},@{n="InstanceStatus";e={$_.Status.Details}}
$instanceState

In [ ]:
$instances|%{Invoke-WebRequest "$($group.HealthCheckProtocol)://$($_.PrivateIpAddress):$($group.Port)$($group.healthCheckPath)"}

In [ ]:
$records|%{Invoke-WebRequest "https://$_" -SkipCertificateCheck}

In [ ]:
Invoke-WebRequest https://demo.soule.aws.sentinel.com

# Appendix

## Identifying API Operation to Cmdlet Mapping

In [ ]:
$apiOperations = @(
    "listHostedZonesByName",
    "listResourceRecordSets",
    "describeLoadBalancers",
    "describeTargetGroups",
    "describeTargetHealth",
    "describeInstances",
    "describeInstanceStatus"
)
foreach($operation in $apiOperations)
{
    Get-AWSCmdletName -ApiOperation $operation
}